In [1]:
from huggingface_hub import login
login()


In [2]:
!pip install -U bitsandbytes
!pip install -U transformers accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

model_id = "meta-llama/Llama-3.1-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",   # Hoặc "bfloat16" nếu GPU hỗ trợ
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model_id, token=True)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    token=True
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, torch_dtype=torch.float16)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:
# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

In [5]:
INSTRUCTION = """
Tôi đã cung cấp một tài liệu ở trên có thể liên quan đến nhiều chủ đề khác nhau.
Dựa trên tài liệu này, hãy tạo ra từ 5 đến 10 cặp câu hỏi và trả lời mang tính thực tế, bao gồm các loại câu hỏi về thời gian, sự kiện, con người, địa điểm hoặc dữ liệu số.
Nếu không đủ số lượng, chỉ cần tạo số câu hỏi bạn tìm được.
Các câu hỏi cần rõ ràng, có thể hiểu độc lập bằng cách bao gồm các từ khóa quan trọng trong tài liệu.
Tránh các câu hỏi mơ hồ như "Phí tham dự sự kiện là bao nhiêu?" mà không có ngữ cảnh.
Ví dụ, thay vào đó hãy hỏi "Phí tham dự sự kiện tại PPG Arena vào ngày 13 tháng 10 là bao nhiêu?"
Trả lời ngắn gọn, không lặp lại câu hỏi hoặc dùng câu hoàn chỉnh.
Ví dụ:
Q: Đại học Quốc Gia Hà Nội được thành lập năm nào? A: 1993
Q: Hiệu trưởng đầu tiên của trường đại học công nghệ là ai? A: Nguyễn Văn Hiệu
"""


In [6]:
import re
import pandas as pd
def extract_qa_pairs(generated_text, doc_id):
    # Use regular expressions to extract all the questions and answers
    questions = re.findall(r'(?:\d*\.\s*)?Q:\s*(.*?)\s*A:', generated_text, re.DOTALL)
    answers = re.findall(r'A:\s*(.*?)(?=\s*(?:\d*\.\s*Q:|Q:|$))', generated_text, re.DOTALL)

    doc_ids = [doc_id] * len(questions)
    # Create a pandas DataFrame from the extracted questions and answers
    df = pd.DataFrame({
        'Doc_id': doc_ids,
        'Question': questions,
        'Answer': answers
    })

    # Display the resulting DataFrame
    return df

In [7]:
import os

output_dir = '../data/annotated'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Created directory: {output_dir}")


Created directory: ../data/annotated


In [8]:
import os
import pandas as pd
from tqdm import tqdm
import datetime
import torch

# Folder chứa các file txt cần xử lý
input_folder = '../content/Test/'
output_dir = '../data/annotated'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Hàm đọc và tiền xử lý từng file txt
def read_preprocess_doc(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    # Bạn có thể thêm bước tiền xử lý ở đây nếu cần
    return text

# Hàm trích xuất Q&A từ text đã sinh
def extract_qa_pairs(generated_text, doc_id):
    import re
    questions = re.findall(r'(?:\d*\.\s*)?Q:\s*(.*?)\s*A:', generated_text, re.DOTALL)
    answers = re.findall(r'A:\s*(.*?)(?=\s*(?:\d*\.\s*Q:|Q:|$))', generated_text, re.DOTALL)

    questions = [q.strip().replace('\n', ' ') for q in questions]
    answers = [a.strip().replace('\n', ' ') for a in answers]

    df = pd.DataFrame({'Doc_id': doc_id, 'Question': questions, 'Answer': answers})
    return df

# Lấy danh sách tất cả file txt trong folder
doc_list = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith('.txt')]

all_qa_df = pd.DataFrame()

for doc in tqdm(doc_list):
    doc_id = os.path.basename(doc)

    print(f"Processing document: {doc_id}")
    content = read_preprocess_doc(doc)
    formatted_input = content + "\n\n" + INSTRUCTION + "\n\nYour answer:"

    tokenized_prompt = tokenizer(formatted_input, return_tensors="pt").to(model.device)
    print(f"Input tokens: {tokenized_prompt.input_ids.size(1)}")

    # Sinh câu trả lời
    messages = [
        {"role": "user", "content": formatted_input},
    ]
    with torch.no_grad():
        result = pipe(messages, max_new_tokens=512)

    # Trích xuất Q&A
    qa_df = extract_qa_pairs(result[0]['generated_text'][1]['content'], doc_id)

    # Append vào DataFrame tổng
    all_qa_df = pd.concat([all_qa_df, qa_df], ignore_index=True)

# Cuối cùng lưu toàn bộ Q&A đã sinh ra vào 1 file duy nhất
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
final_csv_path = os.path.join(output_dir, f'all_generated_qa_pairs_{timestamp}.csv')
all_qa_df.to_csv(final_csv_path, index=False)
print(f"Saved all generated Q&A pairs to {final_csv_path}")


  0%|          | 0/56 [00:00<?, ?it/s]

Processing document: 39.txt
Input tokens: 6232


  2%|▏         | 1/56 [01:04<58:45, 64.11s/it]

Processing document: 35.txt
Input tokens: 1202


  4%|▎         | 2/56 [01:21<33:00, 36.68s/it]

...

Processing document: 14.txt
Input tokens: 4339


 98%|█████████▊| 55/56 [35:16<00:38, 38.00s/it]

Processing document: 58.txt
Input tokens: 4131


100%|██████████| 56/56 [36:08<00:00, 38.71s/it]

Saved all generated Q&A pairs to ../data/annotated/all_generated_qa_pairs_20250520_023517.csv
